# Project for the 'Programming for Data Analysys'
***

### Table of content
[1. Problem statement](#par1)<br>
[2. Simulate the data using``numpy.random`` package](#par2)<br>
[2.1 Simulate perfect process](#par2.1)<br>
[2.1 Add special causes](#par2.2)<br>
[3. Analise the data](#par3)<br>
<hr>

### Import all the packages

In [4]:
# import all the packackes used in the notebook
import numpy as np
import matplotlib.pyplot as plt

In [5]:
# Create the new instance of the numpy psudorandom numbers Generator
rng = np.random.default_rng()

<hr>

<a id="par1"></a>
## 1. Problem statement

## 2. Simulate the data using ```numpy.random``` package

### 2.1 Simulate perfect process

### 2.2 Add special causes

## 3. Analyse the data